# Advanced Exercises: `pytz` Library

This notebook contains a set of **advanced (but not too advanced)** exercises on the `pytz` library, all with worked solutions.

The focus is on:
- Correctly localizing naive datetimes
- Handling DST transitions (ambiguous and non-existent times)
- Converting between time zones safely
- Normalizing datetimes after arithmetic across DST boundaries
- Following best practices (store in UTC, convert at the edges)


## Setup

Run the following cell to import the required modules.


In [1]:
import pytz
from datetime import datetime, timedelta, timezone

from pytz.exceptions import AmbiguousTimeError, NonExistentTimeError

pytz.VERSION

'2025.2'

---
## Exercise 1 – Converting a Meeting Time Across Time Zones

You are given a **naive** datetime representing a team meeting scheduled by a colleague in London:

- Local time (London): `2021-06-15 14:30` (2:30 PM)
- Timezone name: `Europe/London`

1. Localize this naive datetime correctly to London time using `pytz`.
2. Convert the meeting time to the following time zones:
   - `America/New_York`
   - `Asia/Tokyo`
   - `UTC`
3. Print the results in a clear, human-friendly format (e.g. `YYYY-MM-DD HH:MM TZNAME (UTC offset)`).

**Best practice reminder:**

- Do **not** use `replace(tzinfo=...)` to attach non-UTC time zones. Use `localize` instead.


### Solution 1


In [2]:
# 1. Start with the naive datetime
meeting_naive = datetime(2021, 6, 15, 14, 30)
print("Naive meeting time:", meeting_naive)

# 2. Localize to Europe/London correctly
tz_london = pytz.timezone("Europe/London")
meeting_london = tz_london.localize(meeting_naive)
print("London (aware):", meeting_london, "| tzinfo:", meeting_london.tzinfo)

# 3. Convert to other time zones
tz_new_york = pytz.timezone("America/New_York")
tz_tokyo = pytz.timezone("Asia/Tokyo")

meeting_ny = meeting_london.astimezone(tz_new_york)
meeting_tokyo = meeting_london.astimezone(tz_tokyo)
meeting_utc = meeting_london.astimezone(pytz.UTC)

def fmt(dt: datetime) -> str:
    return dt.strftime("%Y-%m-%d %H:%M %Z (%z)")

print("\nConverted times:")
print("London      :", fmt(meeting_london))
print("New York    :", fmt(meeting_ny))
print("Tokyo       :", fmt(meeting_tokyo))
print("UTC         :", fmt(meeting_utc))

# Simple sanity check: all represent the same instant (timestamps equal)
assert meeting_london.timestamp() == meeting_ny.timestamp() == meeting_tokyo.timestamp() == meeting_utc.timestamp()


Naive meeting time: 2021-06-15 14:30:00
London (aware): 2021-06-15 14:30:00+01:00 | tzinfo: Europe/London

Converted times:
London      : 2021-06-15 14:30 BST (+0100)
New York    : 2021-06-15 09:30 EDT (-0400)
Tokyo       : 2021-06-15 22:30 JST (+0900)
UTC         : 2021-06-15 13:30 UTC (+0000)


---
## Exercise 2 – Ambiguous Times at the End of DST

At the end of Daylight Saving Time, some local times occur **twice**. For example, in `America/New_York` in 2020, DST ended on **November 1** at 2:00 AM. The clock went from 1:59:59 AM EDT back to 1:00:00 AM EST.

Consider the naive local time:

- `2020-11-01 01:30` in `America/New_York`

1. Try to localize this naive datetime using `localize()` **without** specifying `is_dst`. Observe what happens.
2. Then correctly create **both** possible aware datetimes:
   - One corresponding to **DST** (`EDT`)
   - One corresponding to **standard time** (`EST`)
   using the `is_dst` parameter.
3. Print both versions, including their UTC offsets, and show that they represent **different instants in time** by comparing their timestamps.


### Solution 2


In [3]:
tz_ny = pytz.timezone("America/New_York")
naive_ambiguous = datetime(2020, 11, 1, 1, 30)
print("Naive time:", naive_ambiguous)

# 1. Try localizing without is_dst
try:
    ambiguous = tz_ny.localize(naive_ambiguous)
    print("Localized without is_dst:", ambiguous)
except AmbiguousTimeError as ex:
    print("AmbiguousTimeError raised (expected at DST end):", ex)

# 2. Correctly disambiguate using is_dst
dt_dst = tz_ny.localize(naive_ambiguous, is_dst=True)   # Treat as DST (EDT)
dt_std = tz_ny.localize(naive_ambiguous, is_dst=False)  # Treat as standard time (EST)

print("\nDisambiguated:")
print("DST version :", dt_dst, "|", dt_dst.strftime("%Y-%m-%d %H:%M %Z (%z)"))
print("STD version :", dt_std, "|", dt_std.strftime("%Y-%m-%d %H:%M %Z (%z)"))

# 3. Show they are different instants in time
print("\nTimestamps:")
print("DST timestamp:", dt_dst.timestamp())
print("STD timestamp:", dt_std.timestamp())
print("Different instants?", dt_dst.timestamp() != dt_std.timestamp())

assert dt_dst != dt_std
assert dt_dst.utcoffset() != dt_std.utcoffset()


Naive time: 2020-11-01 01:30:00
Localized without is_dst: 2020-11-01 01:30:00-05:00

Disambiguated:
DST version : 2020-11-01 01:30:00-04:00 | 2020-11-01 01:30 EDT (-0400)
STD version : 2020-11-01 01:30:00-05:00 | 2020-11-01 01:30 EST (-0500)

Timestamps:
DST timestamp: 1604208600.0
STD timestamp: 1604212200.0
Different instants? True


---
## Exercise 3 – Non-existent Times at the Start of DST

At the **start** of Daylight Saving Time, some local times simply **do not exist**. In `America/New_York` in 2020, clocks jumped from **2:00 AM to 3:00 AM** on March 8, 2020.

Consider the naive local time:

- `2020-03-08 02:30` in `America/New_York`

1. Attempt to localize this naive datetime using `localize()` with **no** `is_dst` parameter. What exception is raised?
2. Catch the exception and handle it by **shifting the time forward** to the first valid time (`03:00`) in that timezone.
3. Print the resulting (corrected) aware datetime and its UTC offset.

This pattern is commonly used when you want to "skip" non-existent times, e.g., for recurring events that occur during the spring forward change.


### Solution 3


In [4]:
tz_ny = pytz.timezone("America/New_York")
naive_nonexistent = datetime(2020, 3, 8, 2, 30)
print("Naive time:", naive_nonexistent)

try:
    bad = tz_ny.localize(naive_nonexistent)
    print("Localized (unexpectedly succeeded):", bad)
except NonExistentTimeError as ex:
    print("NonExistentTimeError raised (expected at DST start):", ex)

# Strategy: shift forward to the earliest valid time (3:00 AM)
corrected_naive = datetime(2020, 3, 8, 3, 0)
corrected = tz_ny.localize(corrected_naive)

print("\nCorrected aware time:", corrected)
print("Formatted         :", corrected.strftime("%Y-%m-%d %H:%M %Z (%z)"))

assert corrected.utcoffset() is not None


Naive time: 2020-03-08 02:30:00
Localized (unexpectedly succeeded): 2020-03-08 02:30:00-05:00

Corrected aware time: 2020-03-08 03:00:00-04:00
Formatted         : 2020-03-08 03:00 EDT (-0400)


---
## Exercise 4 – Arithmetic Across a DST Boundary and Normalization

Suppose you schedule a webinar in `Europe/Berlin` at:

- Start time: `2021-03-28 01:30` (local Berlin time)
- Duration: 2 hours

In 2021, `Europe/Berlin` switched to DST on **March 28**, jumping from 2:00 to 3:00 AM.

1. Localize `2021-03-28 01:30` to `Europe/Berlin` correctly.
2. Add 2 hours using a `timedelta`.
3. Use `tz.normalize()` to ensure the resulting datetime has the correct timezone offset after crossing the DST boundary.
4. Print the start and end times, showing how DST is handled.

**Hint:** Without `normalize`, `pytz` may leave the offset inconsistent after arithmetic on aware datetimes.


### Solution 4


In [5]:
tz_berlin = pytz.timezone("Europe/Berlin")
webinar_start_naive = datetime(2021, 3, 28, 1, 30)

# 1. Localize start time
webinar_start = tz_berlin.localize(webinar_start_naive)
print("Webinar start:", webinar_start, "|", webinar_start.strftime("%Y-%m-%d %H:%M %Z (%z)"))

# 2. Add 2 hours
webinar_end_raw = webinar_start + timedelta(hours=2)
print("\nEnd (before normalize):", webinar_end_raw, "|", webinar_end_raw.strftime("%Y-%m-%d %H:%M %Z (%z)"))

# 3. Normalize to fix offset after DST transition
webinar_end = tz_berlin.normalize(webinar_end_raw)
print("End (after  normalize):", webinar_end, "|", webinar_end.strftime("%Y-%m-%d %H:%M %Z (%z)"))

# Check that we actually crossed into DST
print("\nOffsets:")
print("Start offset:", webinar_start.utcoffset())
print("End   offset:", webinar_end.utcoffset())

assert webinar_start.utcoffset() != webinar_end.utcoffset()
assert (webinar_end - webinar_start).total_seconds() == 2 * 3600


Webinar start: 2021-03-28 01:30:00+01:00 | 2021-03-28 01:30 CET (+0100)

End (before normalize): 2021-03-28 03:30:00+01:00 | 2021-03-28 03:30 CET (+0100)
End (after  normalize): 2021-03-28 04:30:00+02:00 | 2021-03-28 04:30 CEST (+0200)

Offsets:
Start offset: 1:00:00
End   offset: 2:00:00


---
## Exercise 5 – Best Practice: Store in UTC, Display in Local Time

You are designing a simple logging system. The system records events in **UTC** but you want to display them in a user's local time zone.

Suppose you have the following UTC timestamps (naive, coming from some external source):

```python
event_times_utc_naive = [
    datetime(2021, 10, 31, 0, 30),
    datetime(2021, 10, 31, 1, 30),
    datetime(2021, 10, 31, 2, 30),
]
```

1. Convert these **naive** UTC datetimes to **aware** UTC datetimes using a best-practice approach (do not assume they are in some local timezone).
2. Write a helper function `convert_events_to_tz(events_utc, tz_name)` that:
   - Takes a list of UTC-aware datetimes and a timezone name string.
   - Returns a list of converted datetimes in the target timezone.
3. Use this function to convert the events to `Europe/Paris` and `America/Sao_Paulo`.
4. Print the results in a readable table-like format.

Notice how DST might affect the apparent spacing between events in local time.


### Solution 5


In [6]:
# 1. Start with naive UTC datetimes and make them UTC-aware
event_times_utc_naive = [
    datetime(2021, 10, 31, 0, 30),
    datetime(2021, 10, 31, 1, 30),
    datetime(2021, 10, 31, 2, 30),
]

events_utc = [pytz.UTC.localize(dt) for dt in event_times_utc_naive]
print("UTC-aware events:")
for e in events_utc:
    print(" ", e.strftime("%Y-%m-%d %H:%M %Z (%z)"))

# 2. Helper function to convert events from UTC to a target timezone
def convert_events_to_tz(events_utc, tz_name: str):
    tz = pytz.timezone(tz_name)
    converted = [e.astimezone(tz) for e in events_utc]
    return converted

# 3. Convert to Europe/Paris and America/Sao_Paulo
events_paris = convert_events_to_tz(events_utc, "Europe/Paris")
events_sao_paulo = convert_events_to_tz(events_utc, "America/Sao_Paulo")

# 4. Print results
def print_events(label: str, events):
    print(f"\n{label}:")
    for e in events:
        print(" ", e.strftime("%Y-%m-%d %H:%M %Z (%z)"))

print_events("In Europe/Paris", events_paris)
print_events("In America/Sao_Paulo", events_sao_paulo)

# Sanity check: each local event is the same instant as the UTC one
for u, p, s in zip(events_utc, events_paris, events_sao_paulo):
    assert u.timestamp() == p.timestamp() == s.timestamp()


UTC-aware events:
  2021-10-31 00:30 UTC (+0000)
  2021-10-31 01:30 UTC (+0000)
  2021-10-31 02:30 UTC (+0000)

In Europe/Paris:
  2021-10-31 02:30 CEST (+0200)
  2021-10-31 02:30 CET (+0100)
  2021-10-31 03:30 CET (+0100)

In America/Sao_Paulo:
  2021-10-30 21:30 -03 (-0300)
  2021-10-30 22:30 -03 (-0300)
  2021-10-30 23:30 -03 (-0300)


---
## Exercise 6 – Using `fromutc` for Efficient Conversion

In some performance-sensitive code, you have a list of **UTC-aware** datetimes and want to convert them efficiently to a specific timezone using `pytz`'s `fromutc` method.

Given:

```python
tz_delhi = pytz.timezone("Asia/Kolkata")
timestamps_utc = [
    pytz.UTC.localize(datetime(2022, 1, 1, 0, 0)),
    pytz.UTC.localize(datetime(2022, 6, 1, 12, 0)),
]
```

1. Convert these UTC-aware datetimes to `Asia/Kolkata` using **both**:
   - `astimezone(tz_delhi)`
   - `tz_delhi.fromutc(dt_utc)`
2. Show that the results are identical.
3. Comment on when using `fromutc` makes sense and what precondition it assumes about the input datetime.


### Solution 6


In [7]:
tz_delhi = pytz.timezone("Asia/Kolkata")
timestamps_utc = [
    pytz.UTC.localize(datetime(2022, 1, 1, 0, 0)),
    pytz.UTC.localize(datetime(2022, 6, 1, 12, 0)),
]

print("UTC timestamps:")
for dt in timestamps_utc:
    print(" ", dt.strftime("%Y-%m-%d %H:%M %Z (%z)"))

print("\nConverted with astimezone vs fromutc:")

for dt in timestamps_utc:
    # Standard, recommended way:
    via_astimezone = dt.astimezone(tz_delhi)

    # fromutc expects dt.tzinfo is *the same* tz object.
    # We create a datetime whose wall time is UTC, but whose tzinfo is tz_delhi.
    # fromutc will interpret that wall time as UTC and convert to local time.
    dt_for_fromutc = dt.replace(tzinfo=tz_delhi)
    via_fromutc = tz_delhi.fromutc(dt_for_fromutc)

    print("---")
    print("UTC         :", dt.strftime("%Y-%m-%d %H:%M %Z (%z)"))
    print("astimezone  :", via_astimezone.strftime("%Y-%m-%d %H:%M %Z (%z)"))
    print("fromutc     :", via_fromutc.strftime("%Y-%m-%d %H:%M %Z (%z)"))
    assert via_astimezone == via_fromutc

# Comments:
# - fromutc is an internal/low-level method that assumes:
#   * dt.tzinfo is the same timezone (tz_delhi here), and
#   * dt's *clock time* is in UTC.
# - In normal application code you should almost always use astimezone()
#   and never need to call fromutc() directly.


UTC timestamps:
  2022-01-01 00:00 UTC (+0000)
  2022-06-01 12:00 UTC (+0000)

Converted with astimezone vs fromutc:
---
UTC         : 2022-01-01 00:00 UTC (+0000)
astimezone  : 2022-01-01 05:30 IST (+0530)
fromutc     : 2022-01-01 05:30 IST (+0530)
---
UTC         : 2022-06-01 12:00 UTC (+0000)
astimezone  : 2022-06-01 17:30 IST (+0530)
fromutc     : 2022-06-01 17:30 IST (+0530)
